In [31]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ia.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,297207,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,IA,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,297208,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,IA,...,999 Other,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,297209,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,IA,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,NaN
3,297210,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,IA,...,999 Other,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,297211,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,IA,...,999 Other,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,305497,305497,7489088500,3/6/21,736,PPS,VETTER'S INC,3414 Merchant St,Coralville,IA,...,999 Other,4543 Direct Selling Establishments,4543.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,NaN
8291,305498,305498,8071677000,4/8/20,736,PPP,"WEST BRANCH & TIPTON ROOFING COMPANY, INC",323 E College St PO Box 100,WEST BRANCH,IA,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14
8292,305499,305499,8866147009,4/9/20,736,PPP,THE OUTING CLUB,2109 BRADY ST,DAVENPORT,IA,...,999 Other,8134 Civic and Social Organizations,8134.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14
8293,305500,305500,9846667009,4/9/20,736,PPP,DARDIS COMMUNICATIONS,2403 Muddy Creek Lane,CORALVILLE,IA,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",5 to 9,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [ ]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

In [ ]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [4]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/IA_fips_scraped.csv")


In [32]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/IA_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,...,999 Other,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,...,999 Other,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,...,999 Other,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14


In [33]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [34]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [35]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [36]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5525777008,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,5928587102,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,9406087106,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,7964757003,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4414267000,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14
...,...,...,...,...,...
8290,7489088500,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,1.910300e+14
8291,8071677000,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14
8292,8866147009,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14
8293,9846667009,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14


In [37]:
fips_check_scraped_notna['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped_notna['Long'].replace('None', np.nan, inplace=True)

/Users/ciaraspencer/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [38]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

LoanNumber    24
full_add      24
Lat           24
Long          24
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [12]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [13]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('19')] #IA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/1466 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [14]:
geo_id_list =geo_id 

Length Check

In [15]:
print(len(geo_id_list))

1466


Matching FIPS to DataFrame

In [16]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:1466]

In [17]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
2,9406087106,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,[191530051012029]
6,1359287205,"1370 NW 114th Street, Suite 100, CLIVE, IA",41.602953,-93.772291,[191530112062005]
7,9878158707,"5500 Lakeview Pkwy, Davenport, IA",41.577677,-90.508432,[191630129011028]
9,5146417909,"5784 State Highway 175, IDA GROVE, IA",42.350370,-95.482000,[190930903001011]
20,9182667005,"2172 M Ave PO Box 806 0.0, Williamsburg, IA",41.692132,-92.061311,[190959601003067]
...,...,...,...,...,...
8259,7779467008,"27728 Asbury Rd Ste 777, DUBUQUE, IA",42.512599,-90.822421,[190610102014011]
8267,1139808309,"4200 Asbury Rd, Dubuque, IA",42.515592,-90.733902,[190610011031014]
8279,5063367204,"300 walnut street suite 140, des moines, IA",41.586520,-93.621046,[191530051014026]
8290,7489088500,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,[191030003031003]


Remove brackets

In [18]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
2,9406087106,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,191530051012029
6,1359287205,"1370 NW 114th Street, Suite 100, CLIVE, IA",41.602953,-93.772291,191530112062005
7,9878158707,"5500 Lakeview Pkwy, Davenport, IA",41.577677,-90.508432,191630129011028
9,5146417909,"5784 State Highway 175, IDA GROVE, IA",42.350370,-95.482000,190930903001011
20,9182667005,"2172 M Ave PO Box 806 0.0, Williamsburg, IA",41.692132,-92.061311,190959601003067
...,...,...,...,...,...
8259,7779467008,"27728 Asbury Rd Ste 777, DUBUQUE, IA",42.512599,-90.822421,190610102014011
8267,1139808309,"4200 Asbury Rd, Dubuque, IA",42.515592,-90.733902,190610011031014
8279,5063367204,"300 walnut street suite 140, des moines, IA",41.586520,-93.621046,191530051014026
8290,7489088500,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,191030003031003


In [19]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
1508,5553777005,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,NaN
2010,7197977204,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,NaN
2026,9175527000,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,NaN
2369,4553947003,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,NaN
3196,5847707001,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,NaN
3231,4107138504,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,NaN
3560,7398188805,"107 W Main St, Knoxville, IA",41.957237,-77.439880,NaN
3788,4994547007,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,NaN
3873,1160777202,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,NaN
4581,3648238802,"509 E Maple St, Milton, IA",46.047170,-95.297540,NaN


Parse for Merge

In [20]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
2,9406087106,191530051012029
6,1359287205,191530112062005
7,9878158707,191630129011028
9,5146417909,190930903001011
20,9182667005,190959601003067
...,...,...
8259,7779467008,190610102014011
8267,1139808309,190610011031014
8279,5063367204,191530051014026
8290,7489088500,191030003031003


Read in base data

In [22]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ia.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/IA_fips_scraped.csv") 
fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,IA,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,IA,...,999 Other,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,IA,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,NaN
3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,IA,...,999 Other,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,IA,...,999 Other,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14


Merge with MainFrame

In [23]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,IA,...,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14,NaN
1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,IA,...,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14,NaN
2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,IA,...,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,NaN,191530051012029
3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,IA,...,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14,NaN
4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,IA,...,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,8290,305497,7489088500,3/6/21,736,PPS,VETTER'S INC,3414 Merchant St,Coralville,IA,...,4543 Direct Selling Establishments,4543.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,NaN,191030003031003
8291,8291,305498,8071677000,4/8/20,736,PPP,"WEST BRANCH & TIPTON ROOFING COMPANY, INC",323 E College St PO Box 100,WEST BRANCH,IA,...,"2381 Foundation, Structure, and Building Exter...",2381.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14,NaN
8292,8292,305499,8866147009,4/9/20,736,PPP,THE OUTING CLUB,2109 BRADY ST,DAVENPORT,IA,...,8134 Civic and Social Organizations,8134.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14,NaN
8293,8293,305500,9846667009,4/9/20,736,PPP,DARDIS COMMUNICATIONS,2403 Muddy Creek Lane,CORALVILLE,IA,...,"5416 Management, Scientific, and Technical Con...",5416.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",5 to 9,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [24]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,IA,...,3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14,NaN,191670706011036.0
1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,IA,...,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14,NaN,191530111141013.0
2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,IA,...,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,NaN,191530051012029,191530051012029
3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,IA,...,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14,NaN,191630129011028.0
4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,IA,...,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14,NaN,191130107002070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,8290,305497,7489088500,3/6/21,736,PPS,VETTER'S INC,3414 Merchant St,Coralville,IA,...,4543.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,NaN,191030003031003,191030003031003
8291,8291,305498,8071677000,4/8/20,736,PPP,"WEST BRANCH & TIPTON ROOFING COMPANY, INC",323 E College St PO Box 100,WEST BRANCH,IA,...,2381.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14,NaN,190314505003011.0
8292,8292,305499,8866147009,4/9/20,736,PPP,THE OUTING CLUB,2109 BRADY ST,DAVENPORT,IA,...,8134.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14,NaN,191630116003050.0
8293,8293,305500,9846667009,4/9/20,736,PPP,DARDIS COMMUNICATIONS,2403 Muddy Creek Lane,CORALVILLE,IA,...,5416.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",5 to 9,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14,NaN,191030003061000.0


Drop Extra FIPS Columns

In [25]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,IA,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,191670706011036.0
1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,IA,...,999 Other,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,191530111141013.0
2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,IA,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,191530051012029
3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,IA,...,999 Other,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,191630129011028.0
4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,IA,...,999 Other,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,191130107002070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,8290,305497,7489088500,3/6/21,736,PPS,VETTER'S INC,3414 Merchant St,Coralville,IA,...,999 Other,4543 Direct Selling Establishments,4543.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,191030003031003
8291,8291,305498,8071677000,4/8/20,736,PPP,"WEST BRANCH & TIPTON ROOFING COMPANY, INC",323 E College St PO Box 100,WEST BRANCH,IA,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,190314505003011.0
8292,8292,305499,8866147009,4/9/20,736,PPP,THE OUTING CLUB,2109 BRADY ST,DAVENPORT,IA,...,999 Other,8134 Civic and Social Organizations,8134.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,191630116003050.0
8293,8293,305500,9846667009,4/9/20,736,PPP,DARDIS COMMUNICATIONS,2403 Muddy Creek Lane,CORALVILLE,IA,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",5 to 9,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,191030003061000.0


In [26]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
96,96,297303,4488017009,4/3/20,761,PPP,GOLD-EAGLE COOPERATIVE,515 N LOCUST PO BOX 250,GOLDFIELD,IA,...,999 Other,4931 Warehousing and Storage,4931.0,DES MOINES DISTRICT OFFICE,d $2-5 million,250 to 499,"515 N LOCUST PO BOX 250, GOLDFIELD, IA",NaN,NaN,NaN
1508,1508,298715,5553777005,4/5/20,736,PPP,LIME ROCK SPRINGS CO.,10537 HWY 52 N,DUBUQUE,IA,...,999 Other,4249 Miscellaneous Nondurable Goods Merchant W...,4249.0,CEDAR RAPIDS BRANCH OFFICE,"b $350,000-1 million",50 to 99,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,NaN
2010,2010,299217,7197977204,4/28/20,736,PPP,"US NAMEPLATE COMPANY, INC.",2100 HIGHWAY 30 West,MOUNT VERNON,IA,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,CEDAR RAPIDS BRANCH OFFICE,"b $350,000-1 million",50 to 99,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,NaN
2026,2026,299233,9175527000,4/9/20,761,PPP,YOHN CO,309 S. 20TH ST,CLEAR LAKE,IA,...,327 Nonmetallic Mineral Product,3273 Cement and Concrete Product Manufacturing,3273.0,DES MOINES DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,NaN
2369,2369,299576,4553947003,4/3/20,761,PPP,O'DONNELL ACE HARDWARE INC,3469 Highway 65/69 PO BOX 870,CARLISLE,IA,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,DES MOINES DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,NaN
3196,3196,300403,5847707001,4/6/20,736,PPP,ENGLISH VALLEY NURSING CARE CE,150 W WASHINGTON ST PO Box 98,NORTH ENGLISH,IA,...,999 Other,6239 Other Residential Care Facilities,6239.0,CEDAR RAPIDS BRANCH OFFICE,"b $350,000-1 million",50 to 99,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,NaN
3231,3231,300438,4107138504,2/25/21,761,PPP,MIDLAND MANUFACTURING CO. INC,101 E County Line Rd Midland Manufacturing Co Inc,Monroe,IA,...,326 Plastics and Rubber Products,3261 Plastics Product Manufacturing,3261.0,DES MOINES DISTRICT OFFICE,"b $350,000-1 million",20 to 49,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,NaN
3560,3560,300767,7398188805,4/21/21,761,PPP,SOAP CREEK RESOURCES INC,107 W Main St,Knoxville,IA,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,DES MOINES DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"107 W Main St, Knoxville, IA",41.957237,-77.439880,NaN
3788,3788,300995,4994547007,4/4/20,736,PPP,ST. FRANCIS XAVIER SCHOOL,203 2nd 2ND ST,DYERSVILLE,IA,...,999 Other,6111 Elementary and Secondary Schools,6111.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,NaN
3873,3873,301080,1160777202,4/15/20,761,PPP,MOELLER TRUCKING INC.,12 N 36TH ST,DENISON,IA,...,999 Other,4841 General Freight Trucking,4841.0,DES MOINES DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,NaN


Convert to File

In [28]:
fips_IA = fips_merge_drop


In [29]:
fips_IA.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/IA_fips_scraped.csv") 

Review & Compare

In [30]:
fips_IA = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/IA_fips_scraped.csv") 
fips_IA

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,297207,5525777008,4/5/20,761,PPP,"VOGEL PAINT, INC",1110 ALBANY PL SE,ORANGE CITY,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,1,1,297208,5928587102,4/14/20,761,PPP,"ITA GROUP, INC.",4600 WESTOWN PKWY,WEST DES MOINES,...,999 Other,7211 Traveler Accommodation,7211.0,DES MOINES DISTRICT OFFICE,e $5-10 million,500 or more,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,2,2,297209,9406087106,4/15/20,761,PPP,"MITTERA GROUP, INC.","1312 Locust Street, Suite 202",DES MOINES,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,DES MOINES DISTRICT OFFICE,e $5-10 million,250 to 499,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,3,3,297210,7964757003,4/8/20,736,PPP,MCCARTHY-BUSH CORPORATION,5401 VICTORIA AVE,DAVENPORT,...,999 Other,2362 Nonresidential Building Construction,2362.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4,4,297211,4414267000,4/3/20,736,PPP,"MODERN HOLDINGS, INC.",500 Walford Rd SW,CEDAR RAPIDS,...,999 Other,2382 Building Equipment Contractors,2382.0,CEDAR RAPIDS BRANCH OFFICE,e $5-10 million,250 to 499,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,8290,8290,305497,7489088500,3/6/21,736,PPS,VETTER'S INC,3414 Merchant St,Coralville,...,999 Other,4543 Direct Selling Establishments,4543.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,1.910300e+14
8291,8291,8291,305498,8071677000,4/8/20,736,PPP,"WEST BRANCH & TIPTON ROOFING COMPANY, INC",323 E College St PO Box 100,WEST BRANCH,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",10 to 19,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14
8292,8292,8292,305499,8866147009,4/9/20,736,PPP,THE OUTING CLUB,2109 BRADY ST,DAVENPORT,...,999 Other,8134 Civic and Social Organizations,8134.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",50 to 99,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14
8293,8293,8293,305500,9846667009,4/9/20,736,PPP,DARDIS COMMUNICATIONS,2403 Muddy Creek Lane,CORALVILLE,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,CEDAR RAPIDS BRANCH OFFICE,"a $150,000-350,000",5 to 9,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14
